In [ ]:
import requests
import pandas as pd
import os
import re

In [ ]:
rapid_api_key_1 = os.getenv("rapid_api_key_1")
rapid_api_key_2 = os.getenv("rapid_api_key_2")
rapid_api_key_3 = os.getenv("rapid_api_key_3")
rapid_api_key_4 = os.getenv("rapid_api_key_4")
rapid_api_key_5 = os.getenv("rapid_api_key_5")
rapid_api_key_6 = os.getenv("rapid_api_key_6")
rapid_api_key_7 = os.getenv("rapid_api_key_7")
rapid_api_key_8 = os.getenv("rapid_api_key_8")
rapid_api_key_9 = os.getenv("rapid_api_key_9")

rapid_api_keys = [
    rapid_api_key_1, rapid_api_key_2, rapid_api_key_3,
    rapid_api_key_4, rapid_api_key_5, rapid_api_key_6,
    rapid_api_key_7, rapid_api_key_8, rapid_api_key_9
]

In [ ]:
def fetch_coin_dataset():
    tenx_df = pd.read_csv("./dataset/10x_coins.csv")
    non_tenx_df = pd.read_csv("./dataset/not_10x_coins.csv")

    combined_df = pd.concat([tenx_df, non_tenx_df], ignore_index=True)

    return combined_df

def fetch_coin_details(coin_name,coin_dataset):
    coin_name = coin_name.lower()

    # Lower both sides for comparison
    result = coin_dataset[coin_dataset['name'].str.lower().str.contains(coin_name, na=False)]

    if not result.empty:
        return result.iloc[0]['symbol'], result.iloc[0]['screen_name']
    else:
        return "Coin name not found"

In [ ]:
url = "https://twitter241.p.rapidapi.com/search-v2"

def extract_tweets(coin, symbol, handle, start_date, end_date):
    tweet_data = []
    queries = [
        f"'{coin}' since:{start_date} until:{end_date}",
        f"'{symbol}' since:{start_date} until:{end_date}",
        f"'${symbol}' since:{start_date} until:{end_date}",
        f"'#{coin}' since:{start_date} until:{end_date}",
        f"'#{symbol}' since:{start_date} until:{end_date}",
        f"'@{handle}' since:{start_date} until:{end_date}"
    ]

    key_index = 0  # Start with the first API key

    for query in queries:
        bottom_cursor = None
        query_data = []
        while True:
            current_key = rapid_api_keys[key_index % len(rapid_api_keys)]
            key_index += 1

            headers = {
                "x-rapidapi-key": current_key,
                "x-rapidapi-host": "twitter241.p.rapidapi.com"
            }

            params = {
                "type": "Latest",
                "count": "80",
                "query": query
            }
            if bottom_cursor:
                params["cursor"] = bottom_cursor
            try:
                response = requests.get(url, headers=headers, params=params)
                data = response.json()
            except:
                continue

            try:
                entries = data["result"]["timeline"]["instructions"][0]["entries"]
            except (KeyError, IndexError):
                bottom_cursor = data.get("cursor", {}).get("bottom")
                continue

            query_data.extend(entries)

            if len(query_data) >= 150:
                tweet_data.append(query_data)
                print("Length of tweets scraped till now: ",len(tweet_data))
                break

            bottom_cursor = data.get("cursor", {}).get("bottom")
            print("Length of tweets scraped for query: ",len(query_data))

    return tweet_data

    Runner

In [ ]:
coin = "pendle"
start_date = "2023-01-27"
end_date = "2023-12-30"

coin_dataset = fetch_coin_dataset()
symbol, handle = fetch_coin_details(coin,coin_dataset)
extracted_tweets = extract_tweets(coin,symbol,handle,start_date,end_date)

In [ ]:
def parse_tweet_data(extracted_tweets):
    parsed_tweets = []

    for tweet in extracted_tweets:
        try:
            result = tweet["content"]["itemContent"]["tweet_results"]["result"]
            legacy = result.get("legacy", {})
            user = result["core"]["user_results"]["result"]
            user_legacy = user.get("legacy", {})

            tweet_data = {
                "tweet_type": result.get("__typename"),
                "tweet_id": result.get("rest_id"),
                "username": user_legacy.get("screen_name"),
                "tweet_url": f"https://x.com/{user_legacy.get('screen_name')}/status/{result.get('rest_id')}",
                "twitter_url": f"https://twitter.com/{user_legacy.get('screen_name')}/status/{result.get('rest_id')}",
                # "display_text_range": legacy.get("display_text_range"),
                "text": legacy.get("full_text", "")[:legacy.get("display_text_range", [0, 0])[-1]],
                "source": re.sub(r'<[^>]+>', '', result.get("source", "")),
                "retweet_count": legacy.get("retweet_count"),
                "reply_count": legacy.get("reply_count"),
                "favorite_count": legacy.get("favorite_count"),
                "quote_count": legacy.get("quote_count"),
                "view_count": result.get("views", {}).get("count"),
                "created_at": legacy.get("created_at"),
                "lang": legacy.get("lang"),
                "bookmark_count": legacy.get("bookmark_count"),
                "conversation_id": legacy.get("conversation_id_str"),
                "is_reply": "quoted_status_result" in result,
                "is_quote": legacy.get("is_quote_status"),
                "is_retweet": legacy.get("retweeted"),
                "quoted_status": result.get("quoted_status_result", {}),
                "quoted_id": result.get("quoted_status_result", {}).get("result", {}).get("quotedRefResult", {}).get("result", {}).get("rest_id"),
                "quoted_username": result.get("quoted_status_result", {}).get("result", {}).get("core", {}).get("user_results", {}).get("result", {}).get("legacy", {}).get("screen_name"),
                "is_pinned": "pinned_tweet_ids_str" in legacy,
                "author": {
                    "type": user.get("__typename"),
                    "is_blue_verified": user.get("is_blue_verified"),
                    "user_id": user.get("rest_id"),
                    "affiliates_highlighted_label": user.get("affiliates_highlighted_label"),
                    "screen_name": user_legacy.get("screen_name"),
                    "url": f"https://x.com/{user_legacy.get('screen_name')}",
                    "twitter_url": f"https://twitter.com/{user_legacy.get('screen_name')}",
                    "name": user_legacy.get("name"),
                    "is_verified": user_legacy.get("verified"),
                    "profile_picture": user_legacy.get("profile_image_url_https"),
                    "cover_picture": user_legacy.get("profile_banner_url"),
                    "description": user_legacy.get("description"),
                    "location": user_legacy.get("location"),
                    "followers": user_legacy.get("followers_count"),
                    "following": user_legacy.get("friends_count"),
                    "can_dm": user_legacy.get("can_dm"),
                    "can_media_tag": user_legacy.get("can_media_tag"),
                    "created_at": user_legacy.get("created_at"),
                    "entities": user_legacy.get("entities"),
                    "fast_followers_count": user_legacy.get("fast_followers_count"),
                    "favourites_count": user_legacy.get("favourites_count"),
                    "has_custom_timelines": user_legacy.get("has_custom_timelines"),
                    "is_translator": user_legacy.get("is_translator"),
                    "media_count": user_legacy.get("media_count"),
                    "statuses_count": user_legacy.get("statuses_count"),
                    "withheld_in_countries": user_legacy.get("withheld_in_countries"),
                    "possibly_sensitive": user_legacy.get("possibly_sensitive"),
                    "pinned_tweet_ids": user_legacy.get("pinned_tweet_ids_str")
                },
                "media_urls": legacy.get("entities", {}).get("urls", []),
                "is_conversation_controlled": result.get("edit_control", {}).get("is_edit_eligible")
            }

            parsed_tweets.append(tweet_data)

        except Exception as e:
            continue

    return parsed_tweets

In [ ]:
filtered_tweets = []

for i in range(len(extracted_tweets)):
    filtered_tweets.extend(parse_tweet_data(extracted_tweets[i]))

In [ ]:
df = pd.DataFrame(filtered_tweets)

In [ ]:
df.head()

In [ ]:
df.to_csv("coin_tweets.csv",index=False)